## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose start date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import importlib
import get_started_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
#import read_vars_for_separate_models_for_dependent_vars
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

In [2]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'ALL_spots_wo_CB_cancellation_confirmed'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [3]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [4]:
##### Model 0: all variables - prepare data for the model ####
cols_to_use = read_a_combination_of_variables.\
main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_and_didnt_last_month_vars + did_something_last_month_vars


#### get data for the model ###
(data, base_df, df_timeline_all_vars) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [5]:
for model_number in model_numbers[1:]:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
    get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)

# Model 1: posts_on_facebook

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 31558 periods, 2515 subjects, 190 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2515
 number of periods = 31558
  number of events = 190
partial log-likelihood = -1196.17
  time fit was run = 2021-12-21 10:06:27 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Monthly                                              0.86       2.37       0.25             0.37             1.36                 1.45                 3.88
New.email.subscriber.signups.last.month.div10       -0.12       0.89       0.05            -0.22            -0.01                 0.80                 0.99
SpotHopper_site_not_live_yet                         0.41       1.50       0.26            -0.11             0.92                 0.90                 2.51
had_added_events_manually_edited_events_before_...  -0.02       0.98       0.19            -0.40             0.36                 0.67                 1.43
had_added_events_manually_edited_events_last_3_...  -0.53       0.59       0.29            -1.11             0.04                 0.33                 1.04
had_catering_submissions_before_and_didnt_last_...   0.04       1.04       0.27            -0.49             0.57                 0.61                 1.77
had_catering_submissions_last_3_months              -0.53       0.59       0.23            -0.99            -0.07                 0.37                 0.93
had_clicked_emails_before_and_didnt_last_3_months    0.70       2.02       0.19             0.33             1.07                 1.40                 2.91
had_clicked_emails_last_3_months                     0.09       1.10       0.20            -0.31             0.49                 0.74                 1.64
had_emails_sent_manually_scheduled_emails_sent_...   0.14       1.15       0.18            -0.20             0.48                 0.82                 1.62
had_emails_sent_manually_scheduled_emails_sent_...  -0.26       0.77       0.19            -0.63             0.10                 0.53                 1.11
had_online_orders_before_and_didnt_last_month        0.53       1.71       0.34            -0.14             1.21                 0.87                 3.34
had_online_orders_last_month                        -0.35       0.70       0.36            -1.06             0.36                 0.35                 1.43
had_posts_on_facebook_before_and_didnt_last_month    0.45       1.56       0.18             0.10             0.79                 1.11                 2.20
had_posts_on_facebook_last_month                    -0.17       0.85       0.18            -0.52             0.19                 0.60                 1.20
had_qr_code_menu_scans_before_and_didnt_last_month  -0.59       0.55       0.38            -1.33             0.15                 0.26                 1.16
had_qr_code_menu_scans_last_month                   -0.47       0.63       0.23            -0.91            -0.02                 0.40                 0.98
had_tickets_before_and_didnt_last_3_months          -0.71       0.49       0.17            -1.04            -0.37                 0.35                 0.69
had_tickets_last_3_months                           -0.35       0.70       0.19            -0.72             0.01                 0.49                 1.01
had_visited_qrcode_flyers_page_before_and_didnt...  -0.61       0.54       0.30            -1.20            -0.02                 0.30                 0.98
had_visited_qrcode_flyers_page_last_3_months        -0.60       0.55       0.29            -1.17            -0.04                 0.31                 0.97
had_visited_regular_flyers_page_before_and_didn...   0.35       1.42       0.18             0.00           

# Model 2: changed_picture_or_text

...

<lifelines.CoxTimeVaryingFitter: fitted with 31558 periods, 2515 subjects, 190 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2515
 number of periods = 31558
  number of events = 190
partial log-likelihood = -1186.63
  time fit was run = 2021-12-21 11:25:51 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Fb.page.likes.analysis.avg.log2                     -0.04       0.96       0.02            -0.09            -0.00                 0.92                 1.00
Monthly                                              0.87       2.40       0.25             0.38             1.37                 1.46                 3.93
New.email.subscriber.signups.last.month.div10       -0.11       0.90       0.05            -0.21            -0.00                 0.81                 1.00
SpotHopper_site_not_live_yet                         0.42       1.53       0.26            -0.09             0.94                 0.91                 2.55
had_added_events_manually_edited_events_before_...  -0.02       0.98       0.20            -0.41             0.36                 0.67                 1.44
had_added_events_manually_edited_events_last_3_...  -0.50       0.61       0.30            -1.08             0.08                 0.34                 1.08
had_catering_submissions_before_and_didnt_last_...   0.03       1.03       0.27            -0.50             0.57                 0.61                 1.76
had_catering_submissions_last_3_months              -0.54       0.58       0.23            -1.00            -0.08                 0.37                 0.92
had_clicked_emails_before_and_didnt_last_3_months    0.65       1.92       0.19             0.28             1.03                 1.32                 2.79
had_clicked_emails_last_3_months                     0.13       1.14       0.21            -0.28             0.54                 0.76                 1.71
had_number_of_requests_for_new_text_fragment_be...   0.23       1.26       0.18            -0.12             0.58                 0.89                 1.78
had_number_of_requests_for_new_text_fragment_la...   0.57       1.78       0.30            -0.01             1.16                 0.99                 3.19
had_online_orders_before_and_didnt_last_month        0.46       1.58       0.34            -0.21             1.13                 0.81                 3.09
had_online_orders_last_month                        -0.39       0.68       0.36            -1.10             0.31                 0.33                 1.37
had_posts_liked_before_and_didnt_last_3_months       0.31       1.36       0.19            -0.05             0.67                 0.95                 1.96
had_posts_liked_last_3_months                       -0.37       0.69       0.25            -0.85             0.11                 0.43                 1.11
had_posts_on_facebook_before_and_didnt_last_month    0.38       1.46       0.19             0.01             0.75                 1.01                 2.11
had_posts_on_facebook_last_month                    -0.20       0.82       0.19            -0.57             0.17                 0.57                 1.18
had_posts_seen_before_and_didnt_last_3_months        0.34       1.41       0.21            -0.07             0.75                 0.94                 2.12
had_posts_seen_last_3_months                        -0.04       0.96       0.22            -0.47             0.39                 0.62                 1.48
had_qr_code_menu_scans_before_and_didnt_last_month  -0.61       0.54       0.38            -1.35             0.13                 0.26                 1.14
had_qr_code_menu_scans_last_month                   -0.48       0.62       0.23            -0.93           

# Model 33: New.email.subscribers.allinclusive.last.month.div10

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 31558 periods, 2515 subjects, 190 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2515
 number of periods = 31558
  number of events = 190
partial log-likelihood = -1188.27
  time fit was run = 2021-12-21 11:26:49 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Fb.page.likes.analysis.avg.log2                     -0.05       0.96       0.02            -0.09            -0.00                 0.92                 1.00
Monthly                                              0.89       2.44       0.25             0.40             1.38                 1.49                 3.99
New.email.subscribers.allinclusive.last.month.d...  -0.05       0.95       0.03            -0.10            -0.00                 0.90                 1.00
SpotHopper_site_not_live_yet                         0.44       1.55       0.26            -0.08             0.95                 0.93                 2.58
had_added_events_manually_edited_events_before_...  -0.01       0.99       0.20            -0.40             0.37                 0.67                 1.45
had_added_events_manually_edited_events_last_3_...  -0.52       0.60       0.30            -1.10             0.06                 0.33                 1.06
had_catering_submissions_before_and_didnt_last_...   0.03       1.03       0.27            -0.51             0.56                 0.60                 1.75
had_catering_submissions_last_3_months              -0.55       0.58       0.23            -1.00            -0.09                 0.37                 0.92
had_clicked_emails_before_and_didnt_last_3_months    0.66       1.93       0.19             0.28             1.03                 1.33                 2.80
had_clicked_emails_last_3_months                     0.13       1.13       0.21            -0.28             0.53                 0.76                 1.70
had_number_of_requests_for_new_text_fragment_be...   0.23       1.26       0.18            -0.12             0.58                 0.89                 1.78
had_number_of_requests_for_new_text_fragment_la...   0.57       1.77       0.30            -0.01             1.16                 0.99                 3.19
had_posts_liked_before_and_didnt_last_3_months       0.31       1.37       0.18            -0.05             0.67                 0.95                 1.96
had_posts_liked_last_3_months                       -0.37       0.69       0.25            -0.85             0.11                 0.43                 1.11
had_posts_on_facebook_before_and_didnt_last_month    0.37       1.45       0.19             0.00             0.74                 1.00                 2.09
had_posts_on_facebook_last_month                    -0.20       0.82       0.19            -0.57             0.17                 0.57                 1.18
had_posts_seen_before_and_didnt_last_3_months        0.34       1.41       0.21            -0.06             0.75                 0.94                 2.12
had_posts_seen_last_3_months                        -0.04       0.96       0.22            -0.47             0.39                 0.62                 1.48
had_qr_code_menu_scans_before_and_didnt_last_month  -0.58       0.56       0.38            -1.32             0.16                 0.27                 1.17
had_qr_code_menu_scans_last_month                   -0.49       0.62       0.22            -0.92            -0.05                 0.40                 0.95
had_tickets_before_and_didnt_last_3_months          -0.71       0.49       0.17            -1.04            -0.37                 0.35                 0.69
had_tickets_last_3_months                           -0.31       0.74       0.19            -0.67           